# Backgroud
We have achieved 0.89 classification F1-score in our [last notebook](https://github.com/Sylar257/Non-English-language-NLP/blob/master/Vietnamese%20Sentiment%20Analysis.ipynb) which is just a little shy of the best performing model of the [Vietnamese sentiment competition](https://www.aivivn.com/contests/1) top score (F1-score: 0.90).
This notebook follows similar training steps of the [last notebook](https://github.com/Sylar257/Non-English-language-NLP/blob/master/Vietnamese%20Sentiment%20Analysis.ipynb) but tries to build a **Backward** Vietnamese language model for classification.
The idea is, since our input data is not reading from streaming data, we have the freedom to read the text *forawrd* or *backward* resulting in different **feature extraction** for the language classifier. Hence, let's create a backward model and build an **ensemble model** for more robust performance.

In [1]:
%matplotlib inline

from fastai import *
from fastai.text import *

In [2]:
bs=48

In [4]:
Config.data_path()

PosixPath('/home/projectx/.fastai/data')

In [5]:
data_path = Path(os.getcwd())
data_path

PosixPath('/home/projectx/Documents/Non-English sentiment analysis')

In [11]:
# Same deal here, change the default data_path 
# and create all variable for model construct with `_bwd`
lang = 'vi'
name = f'{lang}wiki'
path = data_path/name
dest = path/'docs'
lm_fns = [f'{lang}_wt_bwd', f'{lang}_wt_vocab_bwd']

In [14]:
dest

PosixPath('/home/projectx/Documents/Non-English sentiment analysis/viwiki/docs')

In [15]:
data = (TextList.from_folder(dest)
            .split_by_rand_pct(0.1, seed=42)
            .label_for_lm()
            .databunch(bs=bs, backwards=True))

# data.save(f'{lang}_databunch_bwd')

In [16]:
data.show_batch

<bound method DataBunch.show_batch of TextLMDataBunch;

Train: LabelList (66149 items)
x: LMTextList
xxbos xxmaj thuyết hỗ sinh 
 
  xxmaj thuyết hỗ sinh hay thuyết tương sinh ( " xxmaj xxunk " ) là lý thuyết sinh học mô tả sự tương tác sinh thái giữa hai hoặc nhiều loài trong đó mỗi loài đều được hưởng lợi từ việc này . xxmaj thuyết hỗ sinh được cho là loại tương tác sinh thái phổ biến nhất và nó thường chiếm ưu thế trong hầu hết các cộng đồng trên toàn thế giới . xxmaj thuyết hỗ sinh có thể tương phản với cạnh tranh giữa các quan thể , giống loài , trong đó mỗi loài có kinh nghiệm trong việc tiến kiệm sức lực , năng lượng , và cùng khai thác tài nguyên , hoặc ký sinh trùng , trong đó một loài được hưởng lợi từ " " công sức " " của loài kia . 
  xxmaj thuật ngữ tương sinh đã được xxmaj pierre - xxmaj joseph van xxmaj xxunk giới thiệu trong cuốn sách năm 1876 của ông xxmaj ký sinh trùng và động vật . xxmaj thuyết tương sinh thường bị bó hẹp với hai loại hiện tượng sinh thái khác : hợp 

In [17]:
data.save(f'{lang}_databunch_bwd')

In [18]:
data.path

PosixPath('/home/projectx/Documents/Non-English sentiment analysis/viwiki/docs')

In [ ]:
data = load_data(dest, f'{lang}_databunch_bwd', bs=bs, backwards=True)

In [19]:
learn = language_model_learner(data, AWD_LSTMSTM, drop_mult=1., pretrained=False)

RuntimeError: CUDA error: out of memory